# Worksheet 8.1: Systems of ODEs with Euler's Method

## ENGR& 240: Applied Numerical Methods

### Objectives
- Write a second order ODE as a system of two first order ODEs
- Implement Euler's method for systems of ODEs
- Track numerical solution values through the first few time steps

## Introduction

In this worksheet, we'll explore how to solve higher-order ODEs by converting them to systems of first-order ODEs and using Euler's method.

### Spring-Mass-Damper System

Consider a spring-mass-damper system:

$$m\frac{d^2x}{dt^2} + B\frac{dx}{dt} + kx = F(t)$$

Where:
- $m$ is the mass
- $B$ is the damping coefficient
- $k$ is the spring constant
- $F(t)$ is the external forcing function

For this worksheet, we'll consider a forcing function:
$$F(t) = Ae^{-Ct}\cos(\omega t)$$

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt

## Task 1: Converting a 2nd Order ODE to a System of First Order ODEs

To apply numerical methods like Euler's method, we need to convert our 2nd order ODE into a system of first order ODEs. Follow these steps:

### Step 1: Define New Variables
When working with a 2nd order ODE, we need to introduce new variables to convert it to a system of first order ODEs. For a spring-mass-damper system, what variables would be appropriate? 

Write your definitions here:
* $y_1 = $ ...
* $y_2 = $ ...

### Step 2: Express the First Derivative
Using your definitions from Step 1, express $\frac{dy_1}{dt}$ in terms of your new variables:

$\frac{dy_1}{dt} = $ ...

### Step 3: Express the Second Derivative
Now, use your definitions to rewrite the original ODE:

$$m\frac{d^2x}{dt^2} + B\frac{dx}{dt} + kx = F(t)$$

In terms of your new variables, this becomes:

$$m\frac{dy_2}{dt} + ... + ... = F(t)$$

### Step 4: Solve for $\frac{dy_2}{dt}$
Rearrange the equation from Step 3 to isolate $\frac{dy_2}{dt}$:

$\frac{dy_2}{dt} = $ ...

### Step 5: Write the System of ODEs
Now we have our system of first order ODEs:

$\frac{dy_1}{dt} = $ ...
$\frac{dy_2}{dt} = $ ...

### Task 1 Exercise: Implementing the System Function

Now implement your system of ODEs as a Python function. This function should take:
* The current time `t`
* The current state vector `y` (containing $y_1$ and $y_2$)
* System parameters (`m`, `B`, `k`)
* Forcing function parameters (`A`, `C`, `omega`)

And return the derivatives $\frac{dy_1}{dt}$ and $\frac{dy_2}{dt}$ as an array.

In [ ]:
def spring_mass_damper(t, y, m, B, k, A, C, omega):
    """System of ODEs for spring-mass-damper with damped oscillatory forcing.
    
    Args:
        t (float): Current time
        y (array): Current state [position, velocity]
        m, B, k: System parameters
        A, C, omega: Forcing function parameters
        
    Returns:
        array: Derivatives [dy1/dt, dy2/dt]
    """
    # Extract position and velocity from state vector
    y1 = y[0]  # position
    y2 = y[1]  # velocity
    
    # Calculate forcing function
    F = A * np.exp(-C * t) * np.cos(omega * t)
    
    # Calculate derivatives
    dy1_dt = # TODO: Complete this expression based on your Step 2
    dy2_dt = # TODO: Complete this expression based on your Step 4
    
    return np.array([dy1_dt, dy2_dt])

## Task 2: Implementing Euler's Method for Systems of ODEs

Now we'll implement Euler's method for solving systems of ODEs. The Euler method for a system uses the same basic idea as for a single ODE, but applied to each equation in the system:

$$y_{i+1} = y_i + h \cdot \frac{dy}{dt}(t_i, y_i)$$

Where for a system, $y$ is now a vector containing all state variables, and $\frac{dy}{dt}$ returns a vector of all derivatives.

In [ ]:
def eulersys(dydt, tspan, y0, h, *args):
    """Solve system of ODEs using Euler method.
    
    Args:
        dydt: Function defining the ODE system dydt = f(t,y)
        tspan: [t0, tf] initial and final values
        y0: Initial values for each equation
        h: Step size
        *args: Additional parameters used by dydt
        
    Returns:
        t: Array of time values
        y: Array of solution values (each column is one variable)
    """
    # Input validation
    if not callable(dydt):
        raise ValueError("dydt must be a callable function")
    
    t0, tf = tspan
    y0 = np.array(y0, dtype=float)
    
    # Create time array
    t = np.arange(t0, tf + h/2, h)  # h/2 term handles floating-point issues
    n = len(t)
    
    # Get number of equations from y0
    n_eqn = len(y0)
    
    # Initialize solution array
    y = np.zeros((n, n_eqn))
    y[0, :] = y0
    
    # Implement Euler method for system
    for i in range(n-1):
        dydt_i = dydt(t[i], y[i, :], *args)
        y[i + 1, :] = y[i, :] + dydt_i * h
    
    return t, y

## Task 3: Solving the Spring-Mass-Damper System

Let's solve the spring-mass-damper system and visualize the results.

In [ ]:
# System parameters
m = 1.0       # Mass (kg)
B = 0.5       # Damping coefficient (kg/s)
k = 2.0       # Spring constant (N/m)
A = 1.0       # Forcing amplitude (N)
C = 0.1       # Forcing damping coefficient (1/s)
omega = 1.5   # Forcing frequency (rad/s)

# Simulation parameters
tspan = [0, 20]   # Time span [t0, tf]
y0 = [0.0, 0.0]   # Initial conditions [position, velocity]
h = 0.1           # Step size

# Solve the system using Euler method
t, y = eulersys(spring_mass_damper, tspan, y0, h, m, B, k, A, C, omega)

# Extract position and velocity
position = y[:, 0]
velocity = y[:, 1]

# Calculate the forcing function
forcing = A * np.exp(-C * t) * np.cos(omega * t)

# Plot the results
plt.figure(figsize=(10, 8))

# Position vs time
plt.subplot(3, 1, 1)
plt.plot(t, position, 'b-')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.title('Spring-Mass-Damper System Response')
plt.grid(True)

# Velocity vs time
plt.subplot(3, 1, 2)
plt.plot(t, velocity, 'r-')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.grid(True)

# Forcing function vs time
plt.subplot(3, 1, 3)
plt.plot(t, forcing, 'g-')
plt.xlabel('Time (s)')
plt.ylabel('Force (N)')
plt.grid(True)

plt.tight_layout()
plt.show()

## Task 4: Tracking Variable Values Through the First Steps

To better understand how Euler's method works for systems of ODEs, let's manually track the values of variables through the first few time steps.

### Step-by-Step Calculation

Let's first understand what we need to calculate for each step using Euler's method:

1. Use the current values of $y_1$ and $y_2$ to calculate $\frac{dy_1}{dt}$ and $\frac{dy_2}{dt}$
2. Update $y_1$ and $y_2$ using Euler's formula: $y_{i+1} = y_i + h \cdot \frac{dy}{dt}(t_i, y_i)$
3. Move to the next time step and repeat

### Manual Calculation Exercise

Fill in the table below by manually applying Euler's method for the first three time steps. Use the system parameters:

- m = 1.0 kg
- B = 0.5 kg/s
- k = 2.0 N/m
- A = 1.0 N
- C = 0.1 1/s
- omega = 1.5 rad/s
- h = 0.1 s
- Initial conditions: $y_1(0) = 0$ (position), $y_2(0) = 0$ (velocity)

For the first row (t = 0), the forcing function value is $F(0) = A\cos(0) = 1.0$ N

| t | $y_1$ (position) | $y_2$ (velocity) | $F(t)$ | $\frac{dy_1}{dt}$ | $\frac{dy_2}{dt}$ |
|---|------------------|------------------|--------|-------------------|-------------------|
| 0.0 | 0.0 | 0.0 | 1.0 | ... | ... |
| 0.1 | ... | ... | ... | ... | ... |
| 0.2 | ... | ... | ... | ... | ... |
| 0.3 | ... | ... | ... | ... | ... |

### Hints for Manual Calculation

1. For t = 0.0:
   - We know $y_1(0) = 0$ and $y_2(0) = 0$
   - Calculate $\frac{dy_1}{dt}(0)$ and $\frac{dy_2}{dt}(0)$ using your system equations
   
2. For t = 0.1:
   - Calculate $y_1(0.1) = y_1(0) + h \cdot \frac{dy_1}{dt}(0)$
   - Calculate $y_2(0.1) = y_2(0) + h \cdot \frac{dy_2}{dt}(0)$
   - Calculate $F(0.1) = A e^{-C \cdot 0.1} \cos(\omega \cdot 0.1)$
   - Calculate $\frac{dy_1}{dt}(0.1)$ and $\frac{dy_2}{dt}(0.1)$ using your system equations
   
3. Continue this process for t = 0.2 and t = 0.3

### Check Your Answers

Once you've completed the table manually, run the following code to check your answers:

In [ ]:
# Tracking parameters
num_steps = 3  # Number of steps to track

# Initialize arrays for tracking
t_track = np.zeros(num_steps + 1)
y_track = np.zeros((num_steps + 1, 2))
dydt_track = np.zeros((num_steps + 1, 2))
F_track = np.zeros(num_steps + 1)

# Set initial conditions
t_track[0] = tspan[0]
y_track[0, :] = y0
dydt_track[0, :] = spring_mass_damper(t_track[0], y_track[0, :], m, B, k, A, C, omega)
F_track[0] = A * np.exp(-C * t_track[0]) * np.cos(omega * t_track[0])

# Perform first num_steps steps and track values
for i in range(num_steps):
    # Calculate next time point
    t_track[i+1] = t_track[i] + h
    
    # Calculate next state using Euler's method
    y_track[i+1, :] = y_track[i, :] + dydt_track[i, :] * h
    
    # Calculate derivatives at the new point
    dydt_track[i+1, :] = spring_mass_damper(t_track[i+1], y_track[i+1, :], m, B, k, A, C, omega)
    
    # Calculate forcing function at the new point
    F_track[i+1] = A * np.exp(-C * t_track[i+1]) * np.cos(omega * t_track[i+1])

# Create a table of results
from IPython.display import display, HTML
import pandas as pd

# Format the data
data = {
    't': t_track,
    'y₁ (position)': y_track[:, 0],
    'y₂ (velocity)': y_track[:, 1],
    'F(t)': F_track,
    'dy₁/dt': dydt_track[:, 0],
    'dy₂/dt': dydt_track[:, 1]
}

df = pd.DataFrame(data)
display(HTML(df.to_html(index=False, float_format="%.6f")))

### Step-by-Step Analysis

After completing the table, analyze what's happening in each step:

1. **Initial Conditions** (t = 0.0):
   - How do the initial position and velocity affect the system?
   - What causes the initial acceleration?

2. **First Step** (t = 0.1):
   - Why does the position change (or not change) the way it does?
   - What forces are acting on the system at this point?

3. **Subsequent Steps**:
   - How do the spring force, damping force, and external forcing interact?
   - Can you identify any patterns in how the values change?

## Task 5: Critical Damping Demonstration

Let's explore critical damping, where the damping coefficient is set to $B = 2\sqrt{mk}$. This represents the boundary between oscillatory behavior (underdamped) and non-oscillatory behavior (overdamped).

In [ ]:
# Calculate critical damping coefficient
B_critical = 2 * np.sqrt(m * k)
print(f"Critical damping coefficient B = {B_critical} kg/s")

# System parameters with critical damping
B_under = 0.5 * B_critical    # Underdamped
B_critical = B_critical       # Critically damped
B_over = 2.0 * B_critical     # Overdamped

# Initial conditions for displacement from equilibrium
y0_displaced = [1.0, 0.0]  # Initial position = 1m, Initial velocity = 0

# Solve for each damping scenario
t, y_under = eulersys(spring_mass_damper, tspan, y0_displaced, h, m, B_under, k, 0, 0, 0)  # No forcing
t, y_critical = eulersys(spring_mass_damper, tspan, y0_displaced, h, m, B_critical, k, 0, 0, 0)
t, y_over = eulersys(spring_mass_damper, tspan, y0_displaced, h, m, B_over, k, 0, 0, 0)

# Plot the comparison
plt.figure(figsize=(10, 6))
plt.plot(t, y_under[:, 0], 'b-', label=f'Underdamped (B = {B_under:.2f})')
plt.plot(t, y_critical[:, 0], 'g-', label=f'Critically damped (B = {B_critical:.2f})')
plt.plot(t, y_over[:, 0], 'r-', label=f'Overdamped (B = {B_over:.2f})')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.title('Spring-Mass-Damper: Effect of Damping Ratio')
plt.legend()
plt.grid(True)
plt.show()

## Exercises

1. What are the advantages of converting a higher-order ODE to a system of first-order ODEs for numerical solution?

2. Try different step sizes in the Euler method. How does the accuracy of the solution change? At what point do you start to see numerical instability?
